# Preprocessing der Trainingsdaten

In [1]:
import pandas as pd
import os
import numpy as np

print("Current working directory: " + os.getcwd())
train_data_path = os.path.join(os.getcwd(), "resources\\train_data\\")
print("Train data path: " + train_data_path)

Current working directory: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF
Train data path: C:\Users\Valentin\Documents\10. Semester\WR2\Projekt 3 - FNN CLF\WR-2-3-FFN-CLF\resources\train_data\


## Datensätze laden

In [2]:
df_list = []
param_list = []
train_data_file_names = [name for name in os.listdir(train_data_path) if 'train_data_dump' in name]
parameter_file_names = [name for name in os.listdir(train_data_path) if 'parameter_dump' in name]
print(train_data_file_names)
print(parameter_file_names)

['2022-07-08-09-40-22-train_data_dump.csv', 'collection1_train_data_dump.csv']
['2022-07-08-09-40-22-parameter_dump.csv', 'collection1_parameter_dump.csv']


In [3]:
for ind, (name_train, name_param) in enumerate(zip(train_data_file_names,parameter_file_names)):
    print("Datensatz Nummer " + str(ind+1))
    df_list.append(pd.read_csv(os.path.join(train_data_path, name_train), header = None))
    print(df_list[-1].shape)
    param_list.append(pd.read_csv(os.path.join(train_data_path, name_param), names = ["coefficientType","parameterNames","parameterValues"]))
    print(param_list[-1].shape)
    if param_list[-1].shape[0] > df_list[-1].shape[0]:
        print(param_list[-1].iloc[0,])
        param_list[-1] = param_list[-1].drop(0).reset_index()
        print(param_list[-1].shape)

Datensatz Nummer 1
(30000, 4801)
(30000, 3)
Datensatz Nummer 2
(84698, 4801)
(84699, 3)
coefficientType    coefficientType
parameterNames      parameterNames
parameterValues    parameterValues
Name: 0, dtype: object
(84698, 4)


## Assembliere die verschiedenen Ergebnisse anhand der Koeffizientenfunktion

In [4]:
# df_base = df_list[0]
# paramdf_base = param_list[0]
# 
# for df, param in zip(df_list[1:],param_list[1:]):
#     # Hänge neuen Datensatz an
#     print(df.shape)
#     df_base = pd.concat([df_base,df], axis = 0, ignore_index = True)
#     print(df_base.shape)
#     # Bestimmte alle bisherigen Duplikate
#     duplicate_rows = df_base.duplicated()
#     # Entferne die Duplikate
#     df_base = df_base.loc[~duplicate_rows,]
#     print(df_base.shape)
#     print("----")
#     # Hänge die neue Parameterliste an und entferne sofort die Duplikate
#     paramdf_base = pd.concat([paramdf_base, param], axis = 0, ignore_index = True).loc[~duplicate_rows,]
#     
#     
# #    value_counts_pre_drop += paramdf_base['coefficientType'].value_counts()
# #    paramdf_base = paramdf_base.loc[~duplicate_rows,]
# #    value_counts_post_drop += paramdf_base['coefficientType'].value_counts()
#     
# print("shape without duplicates: ")
# print(df_base.shape)
# print(paramdf_base.shape)

In [5]:
df_base = pd.concat(df_list,axis = 0,ignore_index = True)
paramdf_base = pd.concat(param_list,axis = 0,ignore_index = True)

print(df_base.shape)
print(paramdf_base.shape)
df_base.head()

(114698, 4801)
(114698, 4)


,0,1,2,3,4,5,6,7,8,9,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
del df_list
del param_list

In [7]:
# Zähle Koeffizientenfunktionen
value_counts_pre_drop = paramdf_base['coefficientType'].value_counts()
print(value_counts_pre_drop)

Random Blocks        62011
Blocks               26510
Completely Random    18444
StripesNBlocks        6485
Strip                 1236
Constant                12
Name: coefficientType, dtype: int64


In [8]:
# Schmeiße die Duplikate
duplicate_rows = df_base.duplicated()
print("shape with duplicates: ")
print(df_base.shape)
df_base = df_base.loc[~duplicate_rows,]
paramdf_base = paramdf_base.loc[~duplicate_rows,]
print("shape without duplicates: ")
print(df_base.shape)
print("Percentage of datapoints lost: " + str((100*sum(duplicate_rows)/len(duplicate_rows))) + "\%")
value_counts_post_drop = paramdf_base['coefficientType'].value_counts()
print(value_counts_post_drop)

shape with duplicates: 
(114698, 4801)
shape without duplicates: 
(102644, 4801)
Percentage of datapoints lost: 10.509337564735217\%
Random Blocks        50054
Blocks               26424
Completely Random    18444
StripesNBlocks        6485
Strip                 1236
Constant                 1
Name: coefficientType, dtype: int64


In [9]:
# Anzahl verloren gegangener Trainingspunkten
print(value_counts_post_drop-value_counts_pre_drop)

Random Blocks       -11957
Blocks                 -86
Completely Random        0
StripesNBlocks           0
Strip                    0
Constant               -11
Name: coefficientType, dtype: int64


In [10]:
# value_counts_post_drop = paramdf_base['coefficientType'].value_counts()
# print(value_counts_post_drop)

In [11]:
df_base = pd.concat([paramdf_base,df_base], axis=1)
df_base.head()

,coefficientType,parameterNames,parameterValues,index,0,1,2,3,4,5,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,",NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Random Blocks,"rhoMin,rhoMax,nBlocks,height,heightVariance,wi...","1000000,1,15,8,8,16,16,2,20",NaN,1,1,1,1,1,1,...,0,0,0,0,1,1,1,1,1,0
9,Random Blocks,"rhoMin,rhoMax,nBlocks,height,heightVariance,wi...","1000000,1,15,8,8,16,16,2,20",NaN,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
10,Random Blocks,"rhoMin,rhoMax,nBlocks,height,heightVariance,wi...","1000000,1,15,8,8,16,16,2,20",NaN,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0
11,Random Blocks,"rhoMin,rhoMax,nBlocks,height,heightVariance,wi...","1000000,1,15,8,8,16,16,2,20",NaN,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,0


In [12]:
# Exportiere Trainingsdaten
df_base.iloc[:,3:].to_csv(os.path.join(train_data_path, "collection2_train_data_dump.csv"),index = False, header = False)
# Exportiere zugehörige Parameter
df_base.iloc[:,0:3].to_csv(os.path.join(train_data_path, "collection2_parameter_dump.csv"),index = False, header = True)

### Konstante Koeffizienten

In [13]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random","StripesNBlocks"]
df_constant = df_base.loc[df_base['coefficientType'] == "Constant",]
print(df_constant.shape)

(1, 4805)


In [14]:
df_constant.iloc[:,-1].describe()

count    1.0
mean     0.0
std      NaN
min      0.0
25%      0.0
50%      0.0
75%      0.0
max      0.0
Name: 4800, dtype: float64

### Streifen Koeffizientenfunktion

In [15]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random","StripesNBlocks"]
df_strip = df_base.loc[df_base['coefficientType'] == "Strip",]
print(df_strip.shape)

(1236, 4805)


In [16]:
df_strip.iloc[:,-1].describe()

count    1236.000000
mean        0.472492
std         0.499445
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: 4800, dtype: float64

### Block Koeffizientenfunktion

In [17]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random","StripesNBlocks"]
df_blocks = df_base.loc[df_base['coefficientType'] == "Blocks",]
print(df_blocks.shape)

(26424, 4805)


In [18]:
df_blocks.iloc[:,-1].describe()

count    26424.000000
mean         0.116599
std          0.320947
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: 4800, dtype: float64

### Random Block Koeffizientenfunktion

In [19]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random","StripesNBlocks"]
df_randBlocks = df_base.loc[paramdf_base['coefficientType'] == "Random Blocks",]
print(df_randBlocks.shape)

(50054, 4805)


In [20]:
df_randBlocks.iloc[:,-1].describe()

count    50054.000000
mean         0.056119
std          0.230154
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          1.000000
Name: 4800, dtype: float64

### Random Koeffizientenfunktion

In [21]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random","StripesNBlocks"]
df_random = df_base.loc[paramdf_base['coefficientType'] == "StripesNBlocks",]
print(df_random.shape)

(6485, 4805)


In [22]:
df_random.iloc[:,-1].describe()

count    6485.000000
mean        0.427140
std         0.494701
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: 4800, dtype: float64

### Stripes and Blocks

In [23]:
# coeff_fun_names = ["Constant","Strip","Blocks","Random Blocks","Completely Random","StripesNBlocks"]
df_stripsNBlocks = df_base.loc[paramdf_base['coefficientType'] == "StripesNBlocks",]
print(df_stripsNBlocks.shape)

(6485, 4805)


In [24]:
df_stripsNBlocks.iloc[:,-1].describe()

count    6485.000000
mean        0.427140
std         0.494701
min         0.000000
25%         0.000000
50%         0.000000
75%         1.000000
max         1.000000
Name: 4800, dtype: float64

In [25]:
del df_base

## Datensatz zusammensetzen

In [26]:
randomState = 0
shareConstant = 1
shareStrip = 1
shareBlocks = 0.3
shareRandBlocks = 0.4
shareRand = 0
shareStripsNBlocks = 0.8

In [27]:
df = pd.concat([df_constant.sample(round(df_constant.shape[0] * shareConstant),random_state = randomState),
                df_strip.sample(round(df_strip.shape[0] * shareStrip),random_state = randomState),
                df_blocks.sample(round(df_blocks.shape[0] * shareBlocks),random_state = randomState),
                df_randBlocks.sample(round(df_randBlocks.shape[0] * shareRandBlocks),random_state = randomState),
                df_random.sample(round(df_random.shape[0] * shareRand),random_state = randomState),
                df_stripsNBlocks.sample(round(df_stripsNBlocks.shape[0] * shareStripsNBlocks),random_state = randomState)
               ], axis = 0, ignore_index = True)

print(df.shape)
df.head()

(34374, 4805)


,coefficientType,parameterNames,parameterValues,index,0,1,2,3,4,5,...,4791,4792,4793,4794,4795,4796,4797,4798,4799,4800
0,Constant,"affectedSubdomains,rhoMin,rhoMax,indexShiftx,i...","6,10,1,1000000,0,0,,,",NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Strip,"height,nStrips,rhoMin,rhoMax,indexShifty","3,4,1,1000000,12,,,,",965.0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
2,Strip,"height,nStrips,rhoMin,rhoMax,indexShifty","1,4,1000000,1,6,,,,",157.0,0,0,1,1,1,1,...,0,0,1,1,1,1,0,0,0,1
3,Strip,"height,nStrips,rhoMin,rhoMax,indexShifty","4,3,1000000,1,8,,,,",1161.0,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,Strip,"height,nStrips,rhoMin,rhoMax,indexShifty","2,4,1000000,1,20,,,,",506.0,1,1,0,0,0,0,...,1,1,0,0,0,0,0,1,1,1


## Speichere Daten

In [28]:
# Exportiere Trainingsdaten
df.iloc[:,3:].to_csv(os.path.join(train_data_path, "training_dataset_ver1.csv"),index = False, header = False)
# Exportiere zugehörige Parameter
df.iloc[:,0:3].to_csv(os.path.join(train_data_path, "training_dataset_ver1_parameters.csv"),index = False, header = True)